In [18]:
from datasets import load_dataset
from qdrant_client import models as qmodels
from qdrant_client import QdrantClient

ds = load_dataset("arbml/Hadith")


Using the latest cached version of the dataset since arbml/Hadith couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/linux/.cache/huggingface/datasets/arbml___hadith/default/0.0.0/44ebe1a07005ee6afd1b0fa316522b18e05a3be1 (last modified on Sat Nov 23 13:13:04 2024).


In [19]:
documents = ds['train']['Text']

## Load the model 

In [20]:
from pylate import indexes, models, retrieve

model = models.ColBERT(
    model_name_or_path="answerdotai/answerai-colbert-small-v1",trust_remote_code=True,device='cuda',
)

No sentence-transformers model found with name answerdotai/answerai-colbert-small-v1.
Loaded the ColBERT model from Stanford NLP.


In [21]:
index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="index",
    embedding_size=96,
    override=True,
)

retriever = retrieve.ColBERT(index=index)

In [22]:
index

In [23]:
documents_ids = [str(i) for i in range(len(documents))]

In [24]:
documents_embeddings = model.encode(
    documents,
    batch_size=256*2,
    is_query=False,
    show_progress_bar=True

)


Encoding documents (bs=512):   0%|          | 0/243 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm

# Chunk the data manually
chunk_size = 4000  # Adjust this based on memory limitations
num_chunks = (len(documents_ids) + chunk_size - 1) // chunk_size  # Calculate number of chunks

for i in tqdm(range(num_chunks)):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(documents_ids))
    
    # Retrieve the chunk
    chunk_ids = documents_ids[start_idx:end_idx]
    chunk_embeddings = documents_embeddings[start_idx:end_idx]
    
    # Add to index
    index.add_documents(
        documents_ids=chunk_ids,
        documents_embeddings=chunk_embeddings,
        batch_size=min(chunk_size, len(chunk_ids))  # Ensure batch_size matches chunk size
    )


  0%|          | 0/32 [00:00<?, ?it/s]

Adding documents to the index (bs=4000):   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
f_1 = "حديث جبريل الطويل عن الاسلام"
f_2 = "هل يجوز زيارة القبور؟"
f_3 = "أحاديث عن الصدقة"
f_4 = "هل يوجب المذي الغسل؟"
f_5 = "ماهي صفات المسلم؟"

In [ ]:
h_1_t = "من ترك صلاة مكتوبة متعمدا برئت منه ذمة الله"  # 40165
h_2_t = "من سن في الإسلام سنة حسنة"  # 37492
h_3_t = "الراحمون يرحمهم الرحمن"  # 25311
h_4_t = "مثل المؤنين في توادهم وتراحمهم كمثل الجسد الواحد"  # 36753
h_5_t = "ألا إن في الجسد مضغة إذا صلحت صلح الجسد كله"  # 81731
h_6_t = "إن العبد ليتكلم بالكلمة من سخط الله"  # 27672
h_7_t = "من تشبه بقوم فهو منهم"  # 23973
h_8_t = "اجتنبوا أم الخبائث"  # 5573
h_9_t = "ليس المؤمن باللعان و لا الطعان ولا الفاحش"  # 22751
h_10_t = "فضل العالم علي العابد كفضل القمر"  # 39828

In [ ]:
h_1 = "العهد الذي  بيننا وبينهم الصلاة فمن تركها فقد كفر"  # 78826
h_2 = "من دعا الي الهدي كان له من الأجور مثل أجور متبعه"  # 27900
h_3 = "من لا يرحم الناس لا يرحمه الله"  # 30040
h_4 = "المؤمن للمؤمن كالبنيان يشد بعضه بعضا"  # 37904
h_5 = "إن الله لا ينظر الي صوركم و لكن ينظرالي قلوبكم"  # 29642
h_6 = "من كان يؤمن بالله واليوم الأخر فليقل خيرا أو ليصمت"  # 25437
h_7 = "يحشر المرء مع من يحب"  #
h_8 = "لعن الله الخمر وشاربه وساقيها"  # 24563
h_9 = "سباب المسلم فسوق وقتاله كفر"  # 4037
h_10 = "من سلك طريقا يلتمس فيه علما"  # 27070

In [ ]:
queries_embeddings = model.encode(
    [h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,h_9,h_10],
    batch_size=2,
    is_query=True,
    show_progress_bar=True
)

Encoding queries (bs=2):   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:

scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=10,
    batch_size=32
)
print(scores)

Retrieving documents (bs=32): 100%|██████████| 1/1 [00:06<00:00,  6.75s/it]

[[{'id': '458', 'score': 14.194942474365234}, {'id': '459', 'score': 10.811463356018066}, {'id': '3909', 'score': 10.032829284667969}, {'id': '3040', 'score': 10.029115676879883}, {'id': '5570', 'score': 9.889310836791992}, {'id': '2850', 'score': 9.817706108093262}, {'id': '3591', 'score': 9.805778503417969}, {'id': '2399', 'score': 9.743278503417969}, {'id': '3906', 'score': 9.706753730773926}, {'id': '3039', 'score': 9.705683708190918}], [{'id': '1913', 'score': 12.945530891418457}, {'id': '1969', 'score': 12.238460540771484}, {'id': '4868', 'score': 12.106008529663086}, {'id': '1968', 'score': 11.945481300354004}, {'id': '3115', 'score': 11.88359546661377}, {'id': '845', 'score': 11.669366836547852}, {'id': '1380', 'score': 11.600079536437988}, {'id': '2491', 'score': 11.585163116455078}, {'id': '3088', 'score': 11.32737922668457}, {'id': '3518', 'score': 11.31519889831543}], [{'id': '1200', 'score': 9.190347671508789}, {'id': '1201', 'score': 9.018960952758789}, {'id': '5306', 'sc

In [ ]:
for el in scores[1]:
    print(el['id'])
    print(documents[int(el['id'])])

1913
أخبرنا قتيبة قال حدثنا عبثر عن برد أخي يزيد بن أبي زياد عن المسيب بن رافع قال سمعت البراء بن عازب يقول قال رسول الله صلى الله عليه وسلم من تبع جنازة حتى يصلى عليها كان له من الأجر قيراط ومن مشى مع الجنازة حتى تدفن كان له من الأجر قيراطان والقيراط مثل أحد.
1969
أخبرنا الحسن بن قزعة قال حدثنا مسلمة بن علقمة قال أنبأنا داود عن عامر عن أبي هريرة قال قال رسول الله صلى الله عليه وسلم من تبع جنازة فصلى عليها ثم انصرف فله قيراط من الأجر ومن تبعها فصلى عليها ثم قعد حتى يفرغ من دفنها فله قيراطان من الأجر كل واحد منهما أعظم من أحد.
4868
أخبرنا عبد الحميد بن محمد قال حدثنا مخلد قال حدثنا ابن جريج عن عطاء عن أيمن مولى ابن عمر عن تبيع عن كعب قال من توضأ فأحسن وضوءه ثم شهد صلاة العتمة في جماعة ثم صلى إليها أربعا مثلها يقرأ فيها ويتم ركوعها وسجودها كان له من الأجر مثل ليلة القدر.
1968
أخبرنا محمد بن بشار قال حدثنا محمد بن جعفر عن عوف عن محمد بن سيرين عن أبي هريرة أن رسول الله صلى الله عليه وسلم قال من تبع جنازة رجل مسلم احتسابا فصلى عليها ودفنها فله قيراطان ومن صلى عليها ثم رجع قبل أن تدفن فإنه ي